# Clustering example: Bulge-Disk separation of an Nbody simulation


The initial model is set up with three components: bulge, disk, and dark matter halo.  The positions of the particles are determined according to the profile of each component: Hernquist, exponential, and truncated NFW, respectively.  The model is collisionless, which means there are no hydrodynamics to consider, only gravity.

The velocities of the particles are chosen to be isotropic for the bulge and dark matter halo.  For the disk, the velocities follow the rotational pattern typical of galaxies like the Milky Way.  Likewise, the total mass is chosen to match the Milky Way (10^12 Msun).

The radial velocity dispersion of the disk is set to zero, which is an artificial choice, but it means that the disk is unstable (Toomre stability parameter Q=0).  This is desirable in our case because the disk will easily form substructure when it is evolved which leads to the formation of the spiral arms.  A disk with spiral structure just looks so much cooler (and realistic) than a featureless exponential disk.

The time evolution is carried out via direct force summation using a CUDA package for GPU-accelerated calculations.  The evolution of the model is stopped after 2 Gyr.  The particle data that I provided gives the snapshot at this time.  I did not include the dark matter in the output files because these particles are not observable and are overall less interesting that the baryons in my opinion.

In [ ]:
import numpy as np
import pandas as pd
from data.utils import plot, image, corner   # Custom plotting functions
import os

In [ ]:
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

# Read the data and inspect

In [ ]:
gal = pd.read_csv(os.path.join('data', "diskgalaxy.csv"), sep=" ")  # Read the data as a Pandas DataFrame

In [ ]:
image(gal)

In [ ]:
gal.shape  # 150,000 rows (particles) with 11 columns (features) 

In [ ]:
gal.columns

Evidently, the particle data is given only in cartesian coordinates (positions: x,y,z, velocities: vx,vy,vz)

We also have a particle type `ptype` which tells us the true membership of each particle.  There are 100k disk particles and 50k bulge particles.


In [ ]:
gal.ptype.value_counts()

Visualize the particles belonging to Bulge and Disk, respectively:

In [ ]:
image(gal, gal.ptype==1)  # Shows the true particles of the Disk

In [ ]:
image(gal, gal.ptype==0)  # Shows the true particles of the Bulge

Our goal in this exercise is to use clustering algorithms to separate the bulge and disk.  We will consider K-Means and Gaussian Mixture Models.  In each case, we will attempt to find clusters within the particle data and compare our results to the true particle types given by `ptype`

# K-means: naive usage

Create an instance of the scikit-learn K-Means class with 2 clusters assumed (one for the Bulge, one for the Disk)

In [ ]:
km = KMeans(n_clusters=2)

What data should we pass into K-Means?  Let's naively give all the cartesian data, stacked into one numpy array:

In [ ]:
X = np.stack( (gal.x, gal.y, gal.z, gal.vx, gal.vy, gal.vz), axis=1 )

Now run the algorithm on the data ("fit") and generate the predicted particle types ("predict")

In [ ]:
ptype = km.fit_predict(X)

Compare the predicted types to the true types:

In [ ]:
def accuracy(predicted, true):
    boo = predicted == true
    acc = float(sum(boo))/len(boo)
    return max( acc, 1.-acc )  # the 0 label from K-means could be either bulge or disk

In [ ]:
accuracy(ptype, gal.ptype)

Hmmm... 50% accuracy is very bad.  What does the galaxy image look like when we separate on these predicted ptypes?

In [ ]:
image(gal, ptype==0)

In [ ]:
image(gal, ptype==1)

In [ ]:
# Import the confusion_matrix function
from sklearn.metrics import confusion_matrix
import pylab as plt
import seaborn as sb

# Generate the predictions
#y_pred = decision_tree_classifier.predict(X_test)

# Calculate the confusion matrix
cm = confusion_matrix(ptype, gal.ptype.values)

# Create a DataFrame for plotting
labels = ['elliptical', 'spiral']
df_cm = pd.DataFrame(cm, columns=labels, index=labels)

# Plot the confusion matrix
sb.heatmap(df_cm, annot=True, fmt='g', cmap='Blues', annot_kws={"size": 15})
plt.title('Confusion Matrix', fontsize=20)
plt.xlabel('Predicted', fontsize=15)
plt.xticks(fontsize=13)
plt.ylabel('Actual', fontsize=15)
plt.yticks(fontsize=13)
plt.show()

# K-means: apply to transformed data

We can do a much better job by thinking about physics a bit.  What features distinguish disks from bulges?  Rotation, flattening, or radial extent could all be important.  Let's transform the data into a coordinate system that naturally emphasizes these properties, i.e. cylindrical coordinates.

We will add columns to our DataFrame: radius in the disk plane `R`, azimuthal angle `phi`, radial velocity in the disk plane `vR`, tangential velocity `vphi`

In [ ]:
gal['R'] = np.sqrt( gal.x**2. + gal.y**2. )
gal['phi'] = np.arctan2( gal.y, gal.x )

In [ ]:
gal['vR'] = (gal.x*gal.vx + gal.y*gal.vy)/gal.R
gal['vphi'] = (gal.x*gal.vy - gal.y*gal.vx)/gal.R

In [ ]:
gal.columns

Let us again apply K-means naively to all the data, but this time using the cylindrical coordinates

In [ ]:
coords = [ "R", "phi", "z", "vR", "vphi", "vz" ]
units = [ "kpc", "radians", "kpc", "km/s", "km/s", "km/s" ]

In [ ]:
labs = [ "{} ({})".format(coord,unit) for coord,unit in zip(coords,units) ]
labs

In [ ]:
#X = np.stack( (gal.R, gal.phi, gal.z, gal.vR, gal.vphi, gal.vz), axis=1 )
X = gal[coords].values

In [ ]:
ptype = km.fit_predict(X)

In [ ]:
accuracy(ptype,gal.ptype)

92.8% accuracy is great.  What do the predicted images look like?

In [ ]:
image(gal, ptype==0)

In [ ]:
image(gal, ptype==1)

Fantastic.  We can see in the above image that a small number of the true Bulge particles are predicted to be in the Disk.

But at this point, we don't really understand why this worked so well.  To gain a better understanding, let's look at a corner plot of our data.  It plots the coordinates of all particles against each of the other coordinates.  Lets also overplot the cluster centroids as predicted by K-means.


In [ ]:
cm = confusion_matrix(ptype, gal.ptype.values)

# Create a DataFrame for plotting
labels = ['elliptical', 'spiral']
df_cm = pd.DataFrame(cm, columns=labels, index=labels)

# Plot the confusion matrix
sb.heatmap(df_cm, annot=True, fmt='g', cmap='Blues', annot_kws={"size": 15})
plt.title('Confusion Matrix', fontsize=20)
plt.xlabel('Predicted', fontsize=15)
plt.xticks(fontsize=13)
plt.ylabel('Actual', fontsize=15)
plt.yticks(fontsize=13)
plt.show()

In [ ]:
corner(X, cmap="Greys", labs=labs, stars = km.cluster_centers_)

The goal of K-means is to partition the feature space (i.e. the 6-dimensional space of all positions and velocities) into N clusters, where we have chosen N=2.  These partitions can be visualized by their centroids, which are the stars in the above plot.

A given particle belongs to the cluster/partition which has the nearest centroid.  You can see how this makes sense in the above plot, especially where the centroids are nicely separated.


Let's make the same plot, but this time plot the true disk particles (blue) separate to the true bulge (red) so that we can judge whether the Kmeans predictions make sense intuitively:

In [ ]:
corner(X, mask=gal.ptype==1, split=True, labs=labs, stars = km.cluster_centers_)

This looks good: wherever the true bulge and disk are well-separated (i.e. blue/red regions), the K-means centroids are also separated.  Note that the clusters are nicely separated in only two coordinates: R and vphi.  All other coordinates show nearly overlapping centroids! (i.e. it looks like only one star in some plots, but there are actually two overlapping stars)

In [ ]:


# Calculate the confusion matrix
cm = confusion_matrix(ptype, gal.ptype.values)

# Create a DataFrame for plotting
labels = ['elliptical', 'spiral']
df_cm = pd.DataFrame(cm, columns=labels, index=labels)

# Plot the confusion matrix
sb.heatmap(df_cm, annot=True, fmt='g', cmap='Blues', annot_kws={"size": 15})
plt.title('Confusion Matrix', fontsize=20)
plt.xlabel('Predicted', fontsize=15)
plt.xticks(fontsize=13)
plt.ylabel('Actual', fontsize=15)
plt.yticks(fontsize=13)
plt.show()

# K-means: apply to subsets of the data

Our K-means fit is dominated by the information contained in R and vphi. We can confirm this by repeating the K-means fit with only that data:


In [ ]:
X = np.stack( (gal.R, gal.vphi), axis=1 )
km = KMeans(n_clusters=2, precompute_distances=True)
ptype = km.fit_predict(X)

In [ ]:
accuracy(ptype,gal.ptype)

This gives us the same accuracy as before!  The predicted centroids are also the same:

In [ ]:
labs=["R (kpc)", "vphi (km/s)"]

In [ ]:
corner(X, mask=gal.ptype==1, split=True, labs=labs, stars = km.cluster_centers_)

The above plot shows the true bulge (red) and disk (blue), as before.  Let's now show the predicted clusters given by our K-means fit:

In [ ]:
corner(X, mask=ptype==0, split=True, labs=labs, stars = km.cluster_centers_)

As stated before, a given particle belongs to the cluster with the nearest centroid.  In other words, we can partition the R-vphi space by drawing a line of equidistant points between these centroids.

But that is not what the above plot seems to indicate!  The boundary between the predicted clusters does not appear to be equidistant to the cluster centroids.  What's going on?

Hint: Look at the numerical range of each feature.

We want all features (i.e. positions and velocities) to be treated with equal weight, no matter their numerical range.  To accomplish this, we need to standardize or normalize our features prior to feeding the data to K-means.  (This is standard practice for all machine learning tasks, not just K-means.)

In [ ]:
X = np.stack( (gal.R, gal.vphi), axis=1 )

for k in range(X.shape[1]):
    
    f = X[:,k]
    #X[:,k] = f/np.abs(f).max()   # Rescales each feature to the range [-1,1]
    #X[:,k] = ( f - f.min() )/( f.max() - f.min() )    # Rescale each feature to the range [0,1]  ## BAD result!
    X[:,k] = (f - f.mean())/f.std()  # Normalizes each feature (i.e. mean=0, std=1)
    

In [ ]:
km = KMeans(n_clusters=2, precompute_distances=True)
ptype = km.fit_predict(X)

In [ ]:
accuracy(ptype,gal.ptype)  # We have improved the accuracy slightly

In [ ]:
labs=["R", "vphi"]

Colored by the true ptypes:

In [ ]:
corner(X, mask=gal.ptype==1, split=True, labs=labs, stars = km.cluster_centers_)

Colored by the predicted ptypes:

In [ ]:
corner(X, mask=ptype==1, split=True, labs=labs, stars = km.cluster_centers_ )

Now the boundary looks equidistant to the two centroids, as expected for K-Means.  Comparing the above two plots, we can also see which particles are misclassified.

To see the same plot with equal axes:

In [ ]:
corner(X, mask=ptype==1, split=True, labs=labs, stars = km.cluster_centers_, aspect=None, size=5)

# K-means: adding features

Now that we understand the importance of normalizing our features, let's supply some additional features to K-means.  Will it improve our result?

Our intuition tells us that the z coordinate should be important when distinguishing the Bulge and Disk. (i.e. the disk is much flatter than the Bulge)


In [ ]:
#X = np.stack( (gal.R, gal.vphi, gal.z), axis=1 )
coords = [ "R", "vphi", "z" ]
X = gal[coords].values.copy()    

Normalize the features:

In [ ]:
for k in range(X.shape[1]):
    f = X[:,k]
    X[:,k] = (f - f.mean())/f.std()  # Normalizes each feature (i.e. mean=0, std=1)


In [ ]:
ptype = km.fit_predict(X)

In [ ]:
accuracy(ptype,gal.ptype)

In [ ]:
corner(X, mask=gal.ptype==1, split=True, labs=coords, stars = km.cluster_centers_)

Despite normalizing the z data, we do not see an improvement in the K-Means result.  Could it be that the only important features are R and vphi?

Why isn't z useful?  In the above plots, we see that the distribution in z for the Bulge and Disk have the same mean (0) but quite different scatter - i.e. the disk is flatter.  But K-Means is not able to leverage this information.  Why?


# K-means: let's do some feature engineering

Let's apply our intution.  Partitioning the feature space into distinct clusters is only possible when the centroids of the clusters are well-separated.  We cannot partition two distributions which are co-located.  So let's replace z with a transformed variable - one which will have a different mean for the Bulge and Disk.  Let's use abs(z).

In [ ]:
X = np.stack( (gal.R, gal.vphi, np.abs(gal.z)), axis=1 )

for k in range(X.shape[1]):
    f = X[:,k]
    X[:,k] = (f - f.mean())/f.std()  # Normalizes each feature (i.e. mean=0, std=1)
    

In [ ]:
ptype = km.fit_predict(X)

In [ ]:
accuracy( ptype, gal.ptype )

The true particle memberships:

In [ ]:
corner(X, mask=gal.ptype==1, split=True, labs=["R","vphi","abs(z)"], stars = km.cluster_centers_)

The predicted clusters:

In [ ]:
corner(X, mask=ptype==1, split=True, labs=["R","vphi","abs(z)"], stars = km.cluster_centers_)

Fantastic - the result improves (though only slightly), and we can see that the centroids of our two clusters do show separation in the abs(z) coordinate.

(Note that the boundaries in these plots do not appear sharp.  There is some overlap between the predicted clusters in each panel.  But this is not a problem: these panels shown 2d projections of partitions in 3 dimensions.)

In our final images, we see very few misclassified bulge particles beyond the disk plane:

In [ ]:
image(gal, ptype==1)

In [ ]:
image(gal, ptype==0)

## Principle Component Analysis  (PCA)
<center>
<img src='http://weigend.com/files/teaching/stanford/2008/stanford2008.wikispaces.com/file/view/pca_example.gif'></img></center>
<br><br>

Here we map our dimensions via PCA

In [ ]:
from sklearn.decomposition import PCA

X = np.stack( (gal.R, gal.phi, gal.z, gal.vR, gal.vphi, gal.vz), axis=1 )

pca = PCA(n_components=6)# n_components=6) #n_components=6
x_reduced = pca.fit(X)

### Lets look at the Principle Components

In [ ]:
print('Principle Componenet Vectors \n')
print(pca.components_, '\n')
print('PC Magnitude \n')
print(pca.explained_variance_ratio_, '\n')
print('PC total Variance \n')
print(np.sum(pca.explained_variance_ratio_))

<h2> <center> But that wasn't very userful.  </center></h2>

<h2> <center> Let's now use the PCA to reduce the size of data.  </center></h2>

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)# n_components=6) #n_components=6
x_reduced = pca.fit_transform(X) #  <---  Note the difference

In [ ]:
print('Principle Componenet Vectors \n')
print(pca.components_, '\n')
print('PC Magnitude \n')
print(pca.explained_variance_ratio_, '\n')
print('PC total Variance \n')
print(np.sum(pca.explained_variance_ratio_))

In [ ]:
print('Transformed data shape')
x_reduced.shape

In [ ]:
km = KMeans(n_clusters=2)
ptype = km.fit_predict(x_reduced)

In [ ]:
accuracy(ptype, gal.ptype)

In [ ]:
image(gal, ptype==1)

In [ ]:
image(gal, ptype==0)

In [ ]:
corner(x_reduced, mask=gal.ptype==0, split=True, labs=['PC1', 'PC2'], stars = km.cluster_centers_ ) #, 'PC3', 'PC4'

### What happens if we try more PCAs ?

# Gausian Mixture Models

Will a different clustering algorithm perform better on the same data?

Let's create an instance of a Gaussian Mixture Model.  As before, let's use 2 components:

In [ ]:
gm = GaussianMixture(n_components=2, covariance_type="full") #, max_iter=20, random_state=0)

Using 2 components means the data will be decomposed into two multi-dimensional Gaussians (one for the Bulge, one for the Disk).  The fitting algorithm will find the optimal parameters for these Gaussians, i.e. their means and covariance matrices.

The dimensionality of these two Gaussians will depend on the number of features that we use.  To start, let's keep it simple and use the two important features from before: R and vphi.


In [ ]:
coords = [ "R", "vphi" ]
X = gal[coords].values.copy()

Normalize the features:

In [ ]:
for k in range(X.shape[1]):
    f = X[:,k]
    X[:,k] = (f - f.mean())/f.std()  # Normalizes each feature (i.e. mean=0, std=1)


In [ ]:
gmfit = gm.fit(X)
ptype = gmfit.predict(X)

In [ ]:
accuracy(ptype,gal.ptype)

80% accuracy is quite bad.  The images look worse: the GMM is roughly equivalent to a simple cut in radius!

In [ ]:
image(gal, ptype==0)

In [ ]:
image(gal, ptype==1)

What do the 2-d Gaussian fits look like in the R-vphi plane?  The ellipses visualize each of the two Gaussians in terms of their location (mean) and covariance matrix.

In [ ]:
corner(X, mask=gal.ptype==0, split=True, labs=coords, gmm=gmfit )

For the GMM, a given particle is assigned to the cluster whose Gaussian has greater amplitude at the location of the particle.  Inspecting the above plot, one of the Gaussians dominates at low R values.  That is why the predicted images appear to show a hole in the middle of the disk - almost all particles at low R are assigned to one of the clusters.

The predicted ptypes, for comparison:

In [ ]:
corner(X, mask=ptype==0, split=True, labs=coords, gmm=gmfit )

Let's consider what we've seen.  With only the R-vphi data, K-means creates a nice separation into 2 clusters with 92% accuracy and good visual results.  In contrast, the GMM separates the same data into 2 clusters at much lower accuracy (79%) with a visually poor result.

This failure illustrates an important point in machine learning: even though GMM is a more sophisticated technique than K-means, it is not the right tool for this dataset.  For any machine learning task, we have to understand the structure of our data and then select the appropriate tool.  For our case, the data in the R-vphi plane is structured in a way that a linear partition can separate the clusters nicely, so K-Means is appropriate.

# GMM: adding more data

Let's not give up on GMMs just yet.  Perhaps the result will improve if we provide more data to the algorithm.  Let's try adding z.

Now that we are using 3 features (R, vphi, z), the GMM will fit two 3-dimensional Gaussians to the data.

In [ ]:
coords = [ "R", "vphi", "z" ]
X = gal[coords].values.copy()

In [ ]:
for k in range(X.shape[1]):
    f = X[:,k]
    X[:,k] = (f - f.mean())/f.std()  # Normalizes each feature (i.e. mean=0, std=1)


In [ ]:
gmfit = gm.fit(X)
ptype = gmfit.predict(X)

In [ ]:
accuracy(ptype,gal.ptype)

A much better result, with similar accuracy as K-Means.  Let's check the images:

In [ ]:
image(gal, ptype==0)

In [ ]:
image(gal, ptype==1)

And the corner plots: (True particle types are blue/red, and the ellipses in each panel represent the 2d projections of the 3d covariance matrices)

In [ ]:
corner(X, mask=gal.ptype==1, split=True, labs=coords, gmm=gmfit )

The predicted particle types based on the which Gaussian has higher amplitude at the given location of each particle:

(Note, there is some overlap between the particle clouds in these plots; but that is not unexpected since our Gaussians are 3 dimensional, and each panel is merely a 2d projection.)

In [ ]:
corner(X, mask=ptype==0, split=True, labs=coords )

It is interesting to note that adding z to the data for the GMM made a dramatic improvement of the results, but adding z to K-means did not affect the result (we needed abs(z) instead).  Lesson: know your data, and develop some intuition for which ML tool is appropriate. No one algorithm is better in an absolute sense; each dataset is unique and requires a different tool to gain optimal insight.

# GMM: 3 clusters and full data

For physical reasons, we expected the data to separate into two main clusters, one corresponding to the Bulge and one to the Disk.  But we do not have to limit ourselves to 2 clusters.

Let's use 3 clusters, and let's provide all the data to our GMM.


In [ ]:
gm = GaussianMixture(n_components=3, covariance_type="full",  random_state=0) #, max_iter=20,

In [ ]:
coords = [ "R", "phi", "z", "vR", "vphi", "vz" ]
X = gal[coords].values.copy()

In [ ]:
for k in range(X.shape[1]):
    f = X[:,k]
    X[:,k] = (f - f.mean())/f.std()  # Normalizes each feature (i.e. mean=0, std=1)


In [ ]:
gmfit = gm.fit(X)
ptype = gmfit.predict(X)

In [ ]:
corner(X, mask=gal.ptype==0, split=True, labs=coords, gmm=gmfit )

Inspecting the corner plots, it looks like the GMM has separated the data into a main outer component (disk), and two inner components at small radii (bulge and inner disk).  The images confirm this:  

In [ ]:
image(gal, ptype==0)

In [ ]:
image(gal, ptype==1)

In [ ]:
image(gal, ptype==2)

---

# Miscellaneous and Scraps

**Plotting a mesh, showing a boundary:**

In [ ]:
nbins=100
Rgrid = np.linspace( X[:,0].min(), X[:,0].max(), nbins )
vphigrid = np.linspace( X[:,1].min(), X[:,1].max(), nbins )
xx, yy = np.meshgrid( Rgrid, vphigrid )
pairs = np.stack( ( xx.flatten(), yy.flatten() ), axis=1 )

In [ ]:
pairs_ptype = km.predict(pairs)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
mask = pairs_ptype == 0
plt.plot( pairs[mask,0], pairs[mask,1], '.', alpha=0.1 )
plt.plot( pairs[~mask,0], pairs[~mask,1], '.', alpha=0.1 )
plt.show()


**K-means on radial distance only:**

In [ ]:
X = gal.R.values[:,None]
ptype = km.fit_predict( X )

In [ ]:
image(gal, ptype==0)

In [ ]:
image(gal, ptype==1)

**K-means on tangential velocity only:**

In [ ]:
X = gal.vphi.values[:,None]
km = KMeans(n_clusters=2)
ptype = km.fit_predict(X)

In [ ]:
accuracy(ptype, gal.ptype)

In [ ]:
image(gal, ptype==0)

In [ ]:
image(gal, ptype==1)

**Use K-Means to find substructure within the disk??**
- the result does not seem very meaningful

In [ ]:
disk = gal[ gal.ptype==1 ]


In [ ]:
len(disk)

In [ ]:
image(disk)

In [ ]:
X = np.stack( (disk.R, disk.phi, disk.z, disk.vR, disk.vphi, disk.vz), axis=1 )
#X = np.stack( (disk.vphi, disk.vz), axis=1 )
km = KMeans(n_clusters=3)#, init='random')
pred = km.fit_predict(X)

In [ ]:
image(disk, pred==0)

In [ ]:
image(disk, pred==1)

In [ ]:
image(disk, pred==2)